ACCESSING WEBSITE


In [1]:
import requests
from bs4 import BeautifulSoup

url= 'https://www.iisermohali.ac.in/faculty/people/faculty'


r = requests.get(url)

if r.status_code ==200:
    soup= BeautifulSoup(r.text, 'html.parser')

print(soup.prettify())

<!DOCTYPE html>
<!-- jsn_educare_pro 1.0.3 -->
<html dir="ltr" lang="en-gb">
 <head>
  <meta content="ORyQQJM-VtjL5iC8Wl3Dfcf448W17zlwvFsxJFtQVr" name="_globalsign-domain-verification"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta charset="utf-8">
   <base href="https://www.iisermohali.ac.in/faculty/people/faculty"/>
   <meta content="text, mailto, 
telephone, 0172-2240-ext" name="keywords"/>
   <meta content="mailto: Email@iisermohali.ac.in
Telephone: 0172-2240-Ext.
 
 [text] 
 
 [text] 
 
 [text] 
 
 [text] 
  
 [text]" name="description"/>
   <meta content="Joomla! - Open Source Content Management" name="generator"/>
   <title>
    Faculty - IISER Mohali
   </title>
   <link href="/plugins/content/jw_ts/jw_ts/tmpl/Default/css/template.css" rel="stylesheet"/>
   <link href="/components/com_imageshow/assets/css/style.css" rel="stylesheet"/>
   <link href="/templates/jsn_educare_pro/css/bootstrap.css" rel="stylesheet"/>
   <link href="/plugins/system

EXTRACTING DATA

In [7]:
import pandas as pd

faculty_info = soup.find_all('tr')

data = {'Name': [], 'Link': [], 'Department': [], 'Research Area': [], 'Email': []}

for faculty in faculty_info:
    name_element = faculty.find('a')
    if name_element:
        name = name_element.get_text()
        f_link = name_element['href']
        link = url + f_link
        
        td_elements = faculty.find_all('td')
        
        # Make sure there are enough <td> elements before accessing them
        if len(td_elements) >= 3:
            department = td_elements[1].get_text(strip=True)
            research_area = td_elements[2].get_text(strip=True)
            
            # Check if there are enough <td> elements for email
            if len(td_elements) >= 6:
                email = td_elements[5].get_text(strip=True)
                full_email = email + '@iisermohali.ac.in'

                # Append data to the dictionary
                data['Name'].append(name)
                data['Link'].append(link)
                data['Department'].append(department)
                data['Research Area'].append(research_area)
                data['Email'].append(full_email)

df = pd.DataFrame(data)

df.to_csv('IISER_MOHALI.csv', index=False)

EXTRACTING ADDITIONAL DATA

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

site_url = 'https://www.iisermohali.ac.in'

def fetch_faculty_info(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        texts = [span.get_text(strip=True) for span in soup.find_all('span', style=lambda value: value and "font-family: arial,helvetica,sans-serif;" in value)]
       
        data = {}
        if texts:
            # Extract position before the comma
            position_text = texts[0]
            data['Position'] = position_text.split(',')[0].strip()

            data['Email'] = texts[texts.index('Email') + 1] if 'Email' in texts else 'N/A'
            data['Phone'] = texts[texts.index('Phone') + 1] if 'Phone' in texts else 'N/A'
            data['Fax'] = texts[texts.index('Fax') + 1] if 'Fax' in texts else 'N/A'

            fax_span = soup.find('span', string=data.get('Fax', ''))
            personal_page_tag = fax_span.find_next('a') if fax_span else None
            data['Personal Page'] = personal_page_tag['href'] if personal_page_tag and re.match(r'^http', personal_page_tag['href']) else 'N/A'

            # Research Area and Selected Publications
            if 'Research Area' in texts:
                data['Research Area'] = texts[texts.index('Research Area') + 1]
            if 'Selected Publications' in texts:
                publications_start_index = texts.index('Selected Publications') + 1
                data['Selected Publications'] = texts[publications_start_index:]
    except:
        data = {
            'Position': 'N/A',
            'Email': 'N/A',
            'Phone': 'N/A',
            'Fax': 'N/A',
            'Personal Page': 'N/A',
            'Research Area': 'N/A',
            'Selected Publications': 'N/A'
        }
    return data

response = requests.get(f'{site_url}/faculty/people/faculty')
soup = BeautifulSoup(response.content, 'html.parser')
faculty_rows = soup.find_all('tr')[1:]

# Create an empty DataFrame to store faculty details
df = pd.DataFrame()

for row in faculty_rows:
    columns = row.find_all('td')
    if len(columns) == 6:
        faculty = {}
        name_element = columns[0].find('a')
        faculty['Name'] = name_element.text if name_element else columns[0].text.strip()
        
        # Check if the link already contains the site_url
        if name_element and site_url not in name_element['href']:
            faculty['Link'] = site_url + name_element['href']
        else:
            faculty['Link'] = name_element['href'] if name_element else "No Link Provided"
        
        faculty['Department'] = columns[1].text.strip()
        faculty['Expertise'] = columns[2].text.strip()
        faculty['Office'] = columns[3].text.strip()
        faculty['Phone'] = columns[4].text.strip()
        faculty['Email'] = columns[5].text.strip() + "@iisermohali.ac.in"
        
        # Fetching details from the faculty's personal page
        additional_info = fetch_faculty_info(faculty['Link'])
        faculty.update(additional_info)
        
        # Append the faculty details to the dataframe
        df = pd.concat([df, pd.DataFrame([faculty])], ignore_index=True)


# Save the dataframe to a CSV file
df.to_csv('IISER_MOHALI_I.csv', index=False, encoding='utf-8')


SAVING IN SQLITE

In [ ]:
import pandas as pd
import sqlite3

# Replace 'your_file.csv' and 'your_database.db' with the actual paths/names
csv_file_path = r"C:\Users\Sirsha\Desktop\IDC\project\IISER_MOHALI.csv"
db_file_path = r"C:\Users\Sirsha\Desktop\IDC\project\IISER_MOHALI.db"

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Create a SQLite connection and cursor
conn = sqlite3.connect(db_file_path)
cursor = conn.cursor()

# Write the DataFrame to a SQLite table
df.to_sql('faculty_data', conn, index=False, if_exists='replace')

# Commit changes and close the connection
conn.commit()
conn.close()

print(f"CSV data has been successfully written to {db_file_path} in the 'faculty_data' table.")